# TP 4 : Optimisation et unittesting

## Contexte

De nombreuses applications nécessitent l'utilisation de matrices de très grande taille. Avec par exemple des millions de lignes et de colonnes. C'est nottament le cas en sciences et ingénierie lors de la résolution d'équations aux dérivées partielles (EDPs), par exemple avec la méthode des éléments finis. Par chance, ces très grandes matrices contiennent une large majorité de zéros. C'est ce que l'on appelle des **matrices creuse** (sparse) en anglais.
Ainsi, leur facteur de remplissage est inférieure à 1%. Par exemple, on représente ci-dessous les valeurs non nulles d'une matrice d'éléments finis.
<img src="https://upload.wikimedia.org/wikipedia/commons/8/8a/Finite_element_sparse_matrix.png?uselang=fr" alt="sparse-mat" width="400" height="200">

Puisqu'il y a en fait très peu d'information à stoquer, c'est à dire les éléments non nuls de la matrice, on ne va pas utiliser des tableaux numpy de taille (n,m) (dit "plein") mais une format spécifique. Parmis les plus classiques, on va implémenter le format **COO** (coordinate list) qui n'est pas forcément le plus efficace mais qui présente de bonnes performances. Il se présente de la façon suivante: 
> pour chaque valeur non nulle on stoque le triplet (ligne, colonne, valeur)

Le produit matrice vecteur est une des opérations de base pour tout bibliothèque matricielle. Pour rappel

Si $A=(a_{ij})$ est une matrice de type $(m, n)$ et $\mathbf{b}=(b_{i})$ est un vecteur de type taille $n$, alors leur produit, noté $A\mathbf{b}=\mathbf{c}^T$ est un vecteur de taille $m$ donnée par :

$$\forall i: c_{i} = \sum_{k=1}^n a_{ik}b_{k} = a_{i1}b_{1}+a_{i2}b_{2}+\cdots+ a_{in}b_{n}$$

## Exercice: benchmarker l'implémentation de la matrice COO
Puisque la performance est clé dans ce contexte, nous allons nous efforcer d'obtenir les meilleurs résultats possibles en termes de vitesse. Même si tous les exemples vus dans le cours instrumentent des fonctions, on ne peut pas faire l'économie d'une structure particulière et donc d'une classe. À vous de choisir la meilleure option pour tester et instrumenter vos codes.

> Attention, dans cet exercice on va chercher à "optimiser" notre représentation de matrice. Cependant, n’oubliez pas:
- [Make it work, make it right, make it fast](https://wiki.c2.com/?MakeItWorkMakeItRightMakeItFast) dans cet ordre. On évitera donc l'optimisation prématurée. 
- Faites fonctionner les classes. 
- Mettre en place le benchmark.
- Optimiser 

### Benchmark
Pour tous les exemples ci-dessous, on comparera l'usage mémoire et le temps de calcul.
1. Comparaison avec numpy.ndarray(plein)
    - Créer un matrice (pleine) aléatoire de taille $10000 \times 10000$ et la multiplier avec un vecteur de taille 10000.
    - Faire de même pour une matrice creuse avec un remplissage de 1/1000.
    
1. Pour les matrices creuses:
    - Générer une matrice de dimension $10^6 \times 10^8$ avec $10^7$ entrés non nulles et la multiplier avec un vecteur de taille correspondante.
    - Mesurer une fois l'usage mémoire pour `MatCooPure` et `MatCooNumpy`
    - Comparer les implémentations de la question suivante.

On fera en sorte que les sorties produisent un rapport lisible pour chacun des benchmarks, par exemple en utilisant des f-strings ou un plot.

> *Remarque*: Il peut être judicieux (ou non) de placer ces tests dans des fonctions (!mémoire!)

### Classe(s) matrice COO
> L'utilisation de fichier python séparée semble opportune.

Créer
1. une classe matrice COO en python pur `MatCooPure`
1. une classe matrice COO en python utilisant numpy `MatCooNumpy`
Si vous le souhaitez, vous pouvez programmer une classe mère `MatCoo` pour simplifier les appels. Ce n'est cependant pas nécessaire pour notre benchmark.

Pour les deux classes, vous programmerez: 
- une initialisation à au hasard avec comme argument : (n,m, n_non_nul)
- une initialisation de la matrice identité (à tester formellement)
- un affichage adapté (extrait + dimensions réelles et mémoire) comprenant un test
- un produit matrice vecteur naïf (qui fonctionne) avec test
- un ou des produits matrice vecteur plus rapides. **Pour ce dernier point:** on pourra au choix coder des méthodes **ou** des fonctions dont le premier argument est une instance de ces classes.


In [16]:
import numpy as np
from scipy import sparse

In [12]:
%%timeit
A = np.random.rand(10000,10000)
B = np.random.rand(10000)
print(np.matmul(A,B))

[2452.47448237 2493.31834058 2502.23258951 ... 2440.31668018 2470.77548376
 2476.93407889]
[2502.39218879 2502.57739825 2514.56398701 ... 2522.22198664 2532.50265861
 2517.93985776]
[2471.35736813 2481.80410781 2509.98956995 ... 2512.59063829 2533.95365168
 2508.22071247]
[2521.81351659 2495.22078575 2502.35344564 ... 2500.56754551 2498.80407998
 2530.09714946]
[2492.96892301 2512.44047346 2472.60538535 ... 2476.75869443 2493.07504828
 2489.6121739 ]
[2514.70850913 2541.80871182 2484.9388503  ... 2521.04435688 2483.08140772
 2524.89019686]
[2492.7630369  2544.95474898 2510.04239119 ... 2497.12526516 2493.5942008
 2518.4710751 ]
[2500.60164129 2498.4422801  2485.39894023 ... 2509.09587245 2506.50091486
 2494.32418782]
2.95 s ± 673 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
#%%timeit
S = sparse.random(10000,10000,density = 1/1000).toarray()
B = np.random.rand(10000)
print(S)
print(np.matmul(S,B))

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[2.51774054 1.49932609 2.90922053 ... 0.18978477 2.3191153  3.37138164]


In [17]:
import MatCooPure as COOpy
import MatCooNumpy as COOnp
vect = np.random.rand(10**8)

In [ ]:
%%time
Mpy = COOpy.MatCooPure(10**6, 10**8, 10**7)
print("created")

In [ ]:
%time respy = Mpy.mul_naif(vect)

In [ ]:
print(respy)

In [ ]:
%%time
Mnp = COOnp.MatCooNumpy(10**6,10**8, 10**2)
resnp = Mnp.mul(vect)

In [11]:
import random
%time random.randint(0,1000000)

CPU times: user 41 µs, sys: 2 µs, total: 43 µs
Wall time: 83.4 µs


283341